## Preprocessing

In [ ]:
#pip install tensorflow

In [ ]:
!pip install keras-tuner

In [80]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [81]:
len(application_df)

34299

In [82]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["NAME","STATUS","SPECIAL_CONSIDERATIONS"], axis=1, inplace=True)
application_df.head(5)

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [83]:
# Find null values
for column in application_df:
    print(f"Column {column} has {application_df[column].isnull().sum()} null values")

Column EIN has 0 null values
Column APPLICATION_TYPE has 0 null values
Column AFFILIATION has 0 null values
Column CLASSIFICATION has 0 null values
Column USE_CASE has 0 null values
Column ORGANIZATION has 0 null values
Column INCOME_AMT has 0 null values
Column ASK_AMT has 0 null values
Column IS_SUCCESSFUL has 0 null values


In [84]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                 34299
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [85]:
# Find duplicate entries
print(f"Duplicate entries: {application_df.duplicated().sum()}")

Duplicate entries: 0


In [86]:
#clean_application_df = application_df.drop_duplicates(keep='last')
application_df.drop_duplicates(inplace=True)
len(application_df)

34299

In [87]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [89]:
list(application_counts[application_counts < 200].index)

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [90]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(application_counts[application_counts < 200].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [91]:
## Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [92]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [93]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<200].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [94]:
application_df.head(10)

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
5,10556855,T3,Independent,C1200,Preservation,Trust,0,5000,1
6,10558440,T3,Independent,C1000,Preservation,Trust,100000-499999,31452,1
7,10566033,T3,Independent,C2000,Preservation,Trust,10M-50M,7508025,1
8,10570430,T7,Independent,C1000,ProductDev,Trust,1-9999,94389,1
9,10571689,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0


In [95]:
# Convert categorical data to numeric with `pd.get_dummies`
application_num_df = pd.get_dummies(application_df)
application_num_df.head()

,EIN,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,10520599,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,10531628,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,10547893,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,10553066,6692,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,10556103,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [96]:
# Split our preprocessed data into our features and target arrays
target = application_num_df["IS_SUCCESSFUL"]
data = application_num_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [97]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize the Model

In [98]:
X_train_scaled.shape

(25724, 43)

In [99]:
number_input_features = len(X_train.columns)
number_input_features

43

In [100]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=20,
        step=3), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=20,
            step=3),
            activation=activation))
               
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [101]:
# Import the kerastuner library
import keras_tuner as kt

# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=3)


In [102]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 90 Complete [00h 00m 54s]
val_accuracy: 0.7306122183799744

Best val_accuracy So Far: 0.7337609529495239
Total elapsed time: 00h 22m 19s


In [103]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)  # prints the dictionary of parameters

{'activation': 'tanh', 'first_units': 7, 'num_layers': 2, 'units_0': 4, 'units_1': 13, 'units_2': 7, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 13, 'num_layers': 3, 'units_0': 13, 'units_1': 13, 'units_2': 4, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 3, 'units_0': 7, 'units_1': 10, 'units_2': 10, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


In [104]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    print('---------------------------------')

268/268 - 1s - loss: 0.5545 - accuracy: 0.7338 - 639ms/epoch - 2ms/step
Loss: 0.5545255541801453, Accuracy: 0.7337609529495239
---------------------------------
268/268 - 1s - loss: 0.5497 - accuracy: 0.7334 - 685ms/epoch - 3ms/step
Loss: 0.5497154593467712, Accuracy: 0.7334110736846924
---------------------------------
268/268 - 1s - loss: 0.5561 - accuracy: 0.7333 - 649ms/epoch - 2ms/step
Loss: 0.5560712218284607, Accuracy: 0.7332944869995117
---------------------------------


In [105]:
# Get second best model hyperparameters
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'relu',
 'first_units': 13,
 'num_layers': 3,
 'units_0': 13,
 'units_1': 13,
 'units_2': 4,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [106]:
# Compare the performance to the second-best model
second_model = tuner.get_best_models(2)[1]
model_loss, model_accuracy = second_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5497 - accuracy: 0.7334 - 708ms/epoch - 3ms/step
Loss: 0.5497154593467712, Accuracy: 0.7334110736846924


In [107]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 7,
 'num_layers': 2,
 'units_0': 4,
 'units_1': 13,
 'units_2': 7,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [108]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7338 - 626ms/epoch - 2ms/step
Loss: 0.5545255541801453, Accuracy: 0.7337609529495239


In [110]:
# Export our model to HDF5 file
best_model.save('Models/AlphabetSoupCharity_Optimization3.h5')